In [96]:
pip install amadeus
from amadeus import Client, ResponseError
import requests

# Configurar credenciales de Amadeus
amadeus = Client(
    client_id="LGnvaKKSXMxVvEpVKAGXsJkB9uljfgFb",
    client_secret="94dBn057Hs0ArIcW"
)

def convertir_moneda(monto, moneda_origen, moneda_destino):
    """Convierte el monto de una moneda a otra usando la API de Exchange Rates."""
    url = f"https://api.exchangerate-api.com/v4/latest/{moneda_origen}"
    try:
        response = requests.get(url).json()
        tasa_cambio = response.get("rates", {}).get(moneda_destino, 1)
        return round(float(monto) * tasa_cambio, 2)
    except Exception as e:
        print(f"Error al convertir moneda: {e}")
        return monto  # Retorna el monto original en caso de error

def obtener_codigo_iata(ciudad):
    """Convierte el nombre de una ciudad en su código IATA, buscando tanto ciudades como aeropuertos."""
    try:
        response = amadeus.reference_data.locations.get(keyword=ciudad, subType="CITY,AIRPORT")
        if response.data:
            return response.data[0].get("iataCode")
        else:
            return None
    except ResponseError as error:
        print(f"Error al obtener código IATA: {error}")
        return None

def buscar_vuelos(origen, destino, fecha, adultos):
    """Busca vuelos entre dos ciudades en una fecha específica con una cantidad de pasajeros editable."""
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origen,
            destinationLocationCode=destino,
            departureDate=fecha,
            adults=adultos
        )
        vuelos = response.data if response.data else []
        
        resultados = []
        for vuelo in vuelos:
            itinerario = vuelo.get("itineraries", [])[0].get("segments", [])
            if not itinerario:
                continue
            
            escalas = len(itinerario) - 1
            precio = float(vuelo["price"]["total"])
            moneda = vuelo["price"]["currency"]
            precio_eur = convertir_moneda(precio, moneda, "EUR")
            precio_usd = convertir_moneda(precio, moneda, "USD")
            
            detalles_vuelo = {
                "precio_original": f"{precio} {moneda}",
                "precio_eur": f"{precio_eur} EUR",
                "precio_usd": f"{precio_usd} USD",
                "origen": itinerario[0]["departure"]["iataCode"],
                "destino": itinerario[-1]["arrival"]["iataCode"],
                "hora_salida": itinerario[0]["departure"]["at"],
                "hora_llegada": itinerario[-1]["arrival"]["at"],
                "escalas": escalas,
                "aerolinea": itinerario[0]["carrierCode"],
                "categoria": vuelo["travelerPricings"][0]["fareDetailsBySegment"][0]["cabin"],
            }
            resultados.append(detalles_vuelo)
        
        return resultados
    except ResponseError as error:
        print(f"Error al buscar vuelos: {error}")
        return []

# Ejemplo de uso
if __name__ == "__main__":
    ciudad_origen = input("Ingrese la ciudad de origen: ")
    ciudad_destino = input("Ingrese la ciudad de destino: ")
    fecha = input("Ingrese la fecha de salida (YYYY-MM-DD): ")
    adultos = int(input("Ingrese la cantidad de pasajeros: "))
    
    origen = obtener_codigo_iata(ciudad_origen)
    destino = obtener_codigo_iata(ciudad_destino)
    
    if not origen:
        print(f"No se encontró un código IATA para la ciudad de origen: {ciudad_origen}")
    if not destino:
        print(f"No se encontró un código IATA para la ciudad de destino: {ciudad_destino}")
    
    if origen and destino:
        vuelos = buscar_vuelos(origen, destino, fecha, adultos)
        if vuelos:
            for vuelo in vuelos:
                print(vuelo)
        else:
            print("No se encontraron vuelos para la búsqueda realizada.")
    else:
        print("No se pudo realizar la búsqueda debido a códigos IATA faltantes.")


Ingrese la ciudad de origen:  Madrid
Ingrese la ciudad de destino:  Paris
Ingrese la fecha de salida (YYYY-MM-DD):  2025-06-06
Ingrese la cantidad de pasajeros:  2


{'precio_original': '157.04 EUR', 'precio_eur': '157.04 EUR', 'precio_usd': '171.17 USD', 'origen': 'MAD', 'destino': 'ORY', 'hora_salida': '2025-06-06T14:20:00', 'hora_llegada': '2025-06-06T16:15:00', 'escalas': 0, 'aerolinea': 'UX', 'categoria': 'ECONOMY'}
{'precio_original': '157.04 EUR', 'precio_eur': '157.04 EUR', 'precio_usd': '171.17 USD', 'origen': 'MAD', 'destino': 'ORY', 'hora_salida': '2025-06-06T16:30:00', 'hora_llegada': '2025-06-06T18:25:00', 'escalas': 0, 'aerolinea': 'UX', 'categoria': 'ECONOMY'}
{'precio_original': '201.04 EUR', 'precio_eur': '201.04 EUR', 'precio_usd': '219.13 USD', 'origen': 'MAD', 'destino': 'ORY', 'hora_salida': '2025-06-06T08:00:00', 'hora_llegada': '2025-06-06T09:55:00', 'escalas': 0, 'aerolinea': 'UX', 'categoria': 'ECONOMY'}
{'precio_original': '216.72 EUR', 'precio_eur': '216.72 EUR', 'precio_usd': '236.22 USD', 'origen': 'MAD', 'destino': 'CDG', 'hora_salida': '2025-06-06T18:05:00', 'hora_llegada': '2025-06-06T23:40:00', 'escalas': 1, 'aeroli